In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!pip install -q rasterio
!pip install -q icecream
!pip install -q split-folders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 49.2 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/Khantil/Dump_Detection/dump_detection_shared/yolov7

/content/drive/MyDrive/Khantil/Dump_Detection/dump_detection_shared/yolov7


In [ ]:
import rasterio
import numpy as np
import rasterio
# from rasterio.plot import show
from rasterio.windows import Window
import cv2
import numpy as np
from matplotlib import pyplot as plt
import torch
import json
import glob
import os
from osgeo import gdal
from icecream import ic
from custom_scripts.data_augmentation import *
import splitfolders
from config import *

In [ ]:
class yolo_train():
  def __init__(self):
    mode = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if str(mode) == 'cuda':
      self.device = 0
      print("GPU Found..")
    else:
      self.device = 'cpu'
      print("GPU not found, selected CPU.")
    self.batch_size = batch_size_
    self.epoch = epoch_
    self.im_size = im_size_
    
    self.hyperparameter = hyperparameter_path
    self.configration = configration_path
    self.pretrained_weights = pretrained_weights_path
  def start_training(self, data_file_name, saving_name, model_saving_path):
    name_of_model = saving_name
    data_file = data_file_name
    custom_dir = model_saving_path
    !python3 train.py --device {self.device} --batch-size {self.batch_size} --epochs {self.epoch} --img {self.im_size} {self.im_size} --data {data_file} --hyp {self.hyperparameter} --cfg {self.configration} --weights {self.pretrained_weights} --name {name_of_model} --custom_dir {custom_dir}


In [ ]:
class draw_bbox():
    def __init__(self):
        self.yolo = yolo_train()


    # This Function is For Display Image
    def show1(self, img):
        plt.figure(figsize=(9,13))
        plt.axis("off")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()
  


    # This Function Fetch Only Coordinate From Json File For Further Use
    # Input of this function is GeoJson file

# --------------------------------------------------------------
    def read_coordinates(self, json_path):
        data = json.load(open(json_path))
        final_data = []
        for i in data["features"]:
            tmp = {"id": "",
                  "boxes": ""}
            id_ = i["properties"]["id"]
            box_coordinates = i["geometry"]["coordinates"][0][:4]
            tmp["id"], tmp["boxes"] = id_, box_coordinates
            final_data.append(tmp)
        return final_data



    def saving_coordinates(self, json_path):
        data = json.load(open(json_path))
        final_data = []
        for i in data["features"]:
          tmp = {"id": "",
                  "boxes": ""}
          id_ = i["id"]
          box_coordinates = i["coordinates"][::-1]
          tmp["id"], tmp["boxes"] = id_, box_coordinates
          final_data.append(tmp)
        return final_data

# ---------------------------------------------------------------

    
    # This Function Return Matched Coordinate Between Start Point to End Point
    def get_all_boxes(self, final_list, start_x, end_x, start_y, end_y):
        
        fetched_bbox = []
        for i in final_list:
            tmp = {"id": "",
                "boxes": ""}
            if (i["boxes"][0][0] >= start_x and i["boxes"][2][0] <= end_x) and (i["boxes"][0][1] >= end_y and i["boxes"][2][1] <= start_y):
                tmp["id"], tmp["boxes"] = i["id"], i["boxes"]
                fetched_bbox.append(tmp)  
        return fetched_bbox


    
   
    # This Function Convert Real World Coordinate to image pixel
    # meta will used for transform coordinate to pixel
    def get_img_pixel_from_json(self, fetched_bbox_list, meta):
        converted_data = []
        for i in fetched_bbox_list:
            tmp = {"id": "",
                "boxes": ""}
            tmp["id"] = i["id"]
            dd = []
            for k in i["boxes"]:
                x_coord = k[0]
                y_coord = k[1]

                rowcol = rasterio.transform.rowcol(meta['transform'], xs=x_coord, ys=y_coord)
                y = rowcol[0]
                x = rowcol[1]
                dd.append([x,y])
            tmp["boxes"] = dd
            converted_data.append(tmp)
        return converted_data


    # Convert bbox value to YOLO value
    # Accept Value in touple
    # size = (img_width, img_height)
    # box = (x,x+w, y, y+h)
    def bbox_to_yolo(self, size, box):
        dw = 1./size[0]
        dh = 1./size[1]
        x = (box[0] + box[1])/2.0
        y = (box[2] + box[3])/2.0
        w = box[1] - box[0]
        h = box[3] - box[2]
        x = x*dw
        w = w*dw
        y = y*dh
        h = h*dh
        return (x,y,w,h)
    
    
    # this function convert cropped image in proper image format and set channel for better visulization.
    # this function receive cropped image part and return image with drawed bounding box on image
    def convert_img_array(self, data, final_list, start_x, start_y, step_x, step_y, save_path, txt_number):
        save_path = save_path.replace("images", "")
        save_path = os.path.join(save_path, "labels")
        if not os.path.exists(save_path):
          os.makedirs(save_path)

        flag = False
        end_x = start_x + step_x
        end_y = start_y + step_y
        data = torch.from_numpy(data)
        data=data.permute(1, 2, 0)
        data = data.numpy()
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
        height, width = data.shape[0], data.shape[1]
        for i in final_list:
            if (i["boxes"][3][0] >= start_x) and (i["boxes"][1][0] <= end_x) and (i["boxes"][3][1] >= start_y) and (i["boxes"][1][1] <= end_y):
                x0 = i["boxes"][3][0] - start_x
                y0 = i["boxes"][3][1] - start_y
                x2 = i["boxes"][1][0] - start_x
                y2 = i["boxes"][1][1] - start_y
                bbox_width = x2 - x0
                bbox_height = y2 - y0
                # cv2.rectangle(data, (x0,y0), (x0+bbox_width, y0+bbox_height), (255,0,0), 3)

                x,y,w,h = self.bbox_to_yolo((width, height), (x0, x0+bbox_width, y0, y0+bbox_height))

                file1 = open(f"{save_path}/img_{txt_number}.txt","a")
                file1.write(f"0 {x} {y} {w} {h} \n")
                file1.close()
                if flag == False:
                    flag = True
            else:
                pass
        return data, flag


    
    
    # this function cut images in sub parts and visulize box on images.
    def crop_and_draw_bbox(self, height, width, crop_size, converted_data, dataset, saving_path, show_box):
        cnt = 0
        step = crop_size
        width_cut = crop_size
        height_cut = crop_size
        saving_path = os.path.join(saving_path, "images")
        if not os.path.exists(saving_path):
            os.makedirs(saving_path)


        for col in range(0, height, step):
            if col + step >= height:
                height_cut = height - col

            for row in range(0, width, step):
                if row+width_cut >= width:
                    final_cut = width - row
                    data = dataset.read([1,2,3], window=Window(row, col, final_cut, height_cut))
                    final_img, flag = self.convert_img_array(data, converted_data, row, col, final_cut, height_cut, saving_path, cnt)
                    if flag != False:
                        if show_box == True:
                            self.show1(final_img)
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                        else:
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                    cnt += 1

                else:
                    data = dataset.read([1,2,3], window=Window(row, col, width_cut, height_cut))
                    final_img, flag = self.convert_img_array(data, converted_data, row, col, width_cut, height_cut, saving_path, cnt)
                    if flag != False:
                        if show_box == True:
                            self.show1(final_img)
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                        else:
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                    cnt += 1




    # by default show_bbox = False, it means function will save images to destination folder,
    # if you want to visulize image while running then set it to True. it will show images while running.
    def main(self, tiff_file_path, geojson_file_path, img_saving_path, model_dir, cropping_size = 2500, show_bbox = False, aug = False):
        if not os.path.exists(img_saving_path):
            os.makedirs(img_saving_path)
            print("Saving Directory Not Found.... \n Created Saving Directory.....")
        else:
            print("Saving Directory Found....")
        cnt_ = 0
        tiff_file = tiff_file_path
        ds = gdal.Open(tiff_file)
        dataset = rasterio.open(tiff_file)
        meta_ = dataset.meta

        width = ds.RasterXSize
        height = ds.RasterYSize

        print("Height = ", height, "Width =", width)
        gt = ds.GetGeoTransform()

        minx = gt[0]
        miny = gt[3]
        maxy = gt[3] + width*gt[4] + height*gt[5] 
        maxx = gt[0] + width*gt[1] + height*gt[2]

        
        try:
            final_list = self.read_coordinates(geojson_file_path)
            print("Approaching First Format")  
        except:
            final_list = self.saving_coordinates(geojson_file_path)
            print("Approaching Second Format")
        fetched_bbox = self.get_all_boxes(final_list, minx, maxx, miny, maxy)
        converted_data = self.get_img_pixel_from_json(fetched_bbox, meta_)
        self.crop_and_draw_bbox(height, width, cropping_size, converted_data, dataset, img_saving_path, show_bbox)
        if aug:
            aug = generateData() 

            data_path = os.path.join(img_saving_path, "images")
            image_save_path = os.path.join(img_saving_path, "images/")
            txt_save_path = os.path.join(img_saving_path, "labels/")

            aug.augmentData(data_path, image_save_path, txt_save_path)

        yolo_train_data_dir = "yolo_training_data"
        splitfolders.ratio(img_saving_path, output=f"data/{yolo_train_data_dir}", seed=1337, ratio=(.8, 0.2))

        yaml_file_name = "yolov7_custom.txt"
        with open(f"data/{yaml_file_name}", "w") as file:
          file.write(f"train: data/{yolo_train_data_dir}/train \nval: data/{yolo_train_data_dir}/val \nnc: 1 \nnames: ['pile']")


        rename_path = os.path.join("data", yaml_file_name)
        new_name = rename_path.replace(".txt", ".yaml")
        os.rename(rename_path, new_name)

        model_saving_name = "model1"
        self.yolo.start_training(new_name, model_saving_name, model_dir)





In [ ]:
tiff_file1 = "/content/drive/MyDrive/Khantil/Dump_Detection/dump_detection_shared/data/NamarDaharat2_Orthomosaic_export_TueAug23092831065915.tif"


geo_data = "/content/drive/MyDrive/Khantil/Dump_Detection/dump_detection_shared/geojson/NamarDaharat2_Orthomosaic_export_TueAug23092831065915.geojson"
save_path = "preprocessed_data"
yolo_model_saving_path = "yolo_sample_testing_modeln"


In [ ]:
obj = draw_bbox() 

# default cropping_size = 2500, show_bbox = False, aug(augmentation) = False
obj.main(tiff_file1, geo_data, save_path, yolo_model_saving_path, cropping_size = 2500, show_bbox = False, aug = False) 

GPU Found..
Saving Directory Not Found.... 
 Created Saving Directory.....
Height =  34326 Width = 47417
Approaching Second Format
Max Number of Crops Reached...
Max Limit Reached...


Copying files: 10 files [00:00, 50.12 files/s]


YOLOR 🚀 2022-11-30 torch 1.13.1+cu116 CUDA:0 (A100-SXM4-40GB, 40536.1875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7_custom.yaml', custom_dir='yolo_sample_testing_modeln', data='data/yolov7_custom.yaml', device='0', entity=None, epochs=70, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.custom.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='model1', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='yolo_sample_testing_modeln', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, v5_metric=False, weights='yolov7.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2023-01-20 05:33:34.739128: I tensorflow/core/uti

In [ ]:
# sample testing function
def crop_and_draw_bbox(self, height, width, crop_size, converted_data, dataset, saving_path, show_box):
        cnt = 0
        step = crop_size
        width_cut = crop_size
        height_cut = crop_size

        crop_number = 5
        max_count_flag = False

        saving_path = os.path.join(saving_path, "images")
        if not os.path.exists(saving_path):
            os.makedirs(saving_path)


        for col in range(0, height, step):
            if max_count_flag == True:
              print("Max Limit Reached...")
              break
            else:
              
              if col + step >= height:
                  height_cut = height - col

              for row in range(0, width, step):
                  if row+width_cut >= width:
                      final_cut = width - row
                      data = dataset.read([1,2,3], window=Window(row, col, final_cut, height_cut))
                      final_img, flag = self.convert_img_array(data, converted_data, row, col, final_cut, height_cut, saving_path, cnt)
                      if flag != False:
                          if show_box == True:
                              self.show1(final_img)
                              cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                          else:
                              cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                      if cnt == crop_number:
                          print("Max Number of Crops Reached...")
                          max_count_flag = True
                          break
                      cnt += 1

                  else:
                      data = dataset.read([1,2,3], window=Window(row, col, width_cut, height_cut))
                      final_img, flag = self.convert_img_array(data, converted_data, row, col, width_cut, height_cut, saving_path, cnt)
                      if flag != False:
                          if show_box == True:
                              self.show1(final_img)
                              cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                          else:
                              cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                      if cnt == crop_number:
                          print("Max Number of Crops Reached...")
                          max_count_flag = True
                          break
                      cnt += 1

In [ ]:
# original function 
def crop_and_draw_bbox(self, height, width, crop_size, converted_data, dataset, saving_path, show_box):
        cnt = 0
        step = crop_size
        width_cut = crop_size
        height_cut = crop_size
        saving_path = os.path.join(saving_path, "images")
        if not os.path.exists(saving_path):
            os.makedirs(saving_path)


        for col in range(0, height, step):
            if col + step >= height:
                height_cut = height - col

            for row in range(0, width, step):
                if row+width_cut >= width:
                    final_cut = width - row
                    data = dataset.read([1,2,3], window=Window(row, col, final_cut, height_cut))
                    final_img, flag = self.convert_img_array(data, converted_data, row, col, final_cut, height_cut, saving_path, cnt)
                    if flag != False:
                        if show_box == True:
                            self.show1(final_img)
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                        else:
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                    cnt += 1

                else:
                    data = dataset.read([1,2,3], window=Window(row, col, width_cut, height_cut))
                    final_img, flag = self.convert_img_array(data, converted_data, row, col, width_cut, height_cut, saving_path, cnt)
                    if flag != False:
                        if show_box == True:
                            self.show1(final_img)
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                        else:
                            cv2.imwrite(f'{saving_path}/img_{cnt}.png', final_img)
                    cnt += 1

In [ ]:
# stack link
# https://stackoverflow.com/questions/74372636/indices-should-be-either-on-cpu-or-on-the-same-device-as-the-indexed-tensor